### Import basic libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

### Load in the dataframe

In [ ]:
df = pd.read_pickle('./main_df.pkl')
print(df.shape)
df.head()

## Data preprocessing

### Get all of positive cases + the same number of negative cases randomly

In [ ]:
# get all the positive cases
learning_df = df[df['y'] == 1]
# get the same amount of negative cases RANDOMLY
negative_learning_df = df[df['y'] == 0].sample(n=len(df[df['y'] == 1].index))

learning_df = learning_df.append(negative_learning_df)
print(learning_df.shape)
learning_df.head()

### Encode categorical values

*Note: encoding not needed because training is not based on virus and host names*

In [ ]:
'''
transformed_data = pd.get_dummies(filled_df, columns=['virus', 'host'])
transformed_data
'''

### Extract X and y arrays

In [ ]:
X = learning_df.drop(['virus', 'host', 'group','y'], axis=1).values
y = learning_df['y'].values

print(f'shape of X: {X.shape}')
print(f'len(y): {len(y)}')

### test_train_split

In [ ]:
# SIMPLE SPLIT
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

### Feature scaling (if necessary) 

### Train the Random Forst Classifier

In [ ]:
# ONLY FOR SIMPLE SPLIT USE
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)

### Predict Test set results

In [ ]:
y_pred = classifier.predict(X_test)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

## Score the model

### Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

### ROC curve

In [ ]:
import sklearn.metrics as metrics
# calculate the fpr and tpr for all thresholds of the classification
probs = classifier.predict_proba(X_test)
preds = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

*Note: conclusion: ROC curve is a bad method to evaluate this kind of data*

### Precision-recall curve

In [ ]:
#TODO implement this
from sklearn.metrics import PrecisionRecallDisplay
display = PrecisionRecallDisplay.from_predictions(y_test, y_pred, name="r_forest")
_ = display.ax_.set_title("r_forest (10 estimators) precision-recall curve")

In [ ]:
display = PrecisionRecallDisplay.from_estimator(
    classifier, X_test, y_test, name="r_forest"
)
_ = display.ax_.set_title("r_forest (10 estimators) Precision-Recall curve")

## F1 score

In [ ]:
print(cm)
accuracy_score(y_test, y_pred)

### Calculate precision and recall

In [ ]:
from sklearn.metrics import precision_score, recall_score
print('Precision is: ', precision_score(y_test, y_pred))
print('Recall is: ', recall_score(y_test, y_pred))

### F1 score for this model 

In [ ]:
from sklearn.metrics import f1_score
print(f'F1 is: {f1_score(y_test, y_pred)}')

*Note: the model still has a very good score*

*Ideas as to why:* 
1. *Now there is enough data to train the model - ca. 16,5k positive cases + 16,5k randomly chosen negative ones*
    * _to get learning_df closer to natural, a different method (learning for unbalanced set - ___stratified sampling___) will probably be needed_ 
2. *Still huge host and phage in-group similarity (hosts often with only a few nucleotide difference in genome) - not adressed!* 
3. *Randomly chosen 'good' negative cases?*

# Training using cross validation

In [ ]:
# Leave one out split
from sklearn.model_selection import LeaveOneOut
cv = LeaveOneOut()

Explicit tqdm handling (better visualisaition)

*Watch out -* ***long!***

In [ ]:
'''
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from tqdm import tqdm
# enumerate splits
y_true, y_pred = list(), list()
pbar = tqdm(total=len(X))

for train_ix, test_ix in cv.split(X):
    # split data
    X_train, X_test = X[train_ix, :], X[test_ix, :]
    y_train, y_test = y[train_ix], y[test_ix]
    # fit model
    model = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state=1)
    model.fit(X_train, y_train)
    # evaluate model
    yhat = model.predict(X_test)
    # store
    y_true.append(y_test[0])
    y_pred.append(yhat[0])
    pbar.update(1)
pbar.close()
# calculate accuracy
acc = accuracy_score(y_true, y_pred)
print(f'Accuracy: {acc:.3f}')
'''

Using cross_val_score function

*Watch out -* ***long and hard to stop!***

In [ ]:
'''
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
# create loocv procedure
cv = LeaveOneOut()
# create model
model = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state=1)
# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
# report performance
print(f'Accuracy: {np.mean(scores):.3f} ({np.std(scores):.3f})')
'''